<h1><center>Boosting</center></h1>
# 1.Basic
Boosting is a greedy algorithm for fitting adaptive basis function models of the form below
$$
g\left[\mu(\vec{x})\right]=w_0+\sum_{m=1}^Mw_mf_m(\vec{x})
$$
where the $f_m$ are generated by an algorithm called a **weak learner**. The **boosting** works by training the weak learner sequentially to a modified versions of the data.This weak learner can be any classification or regression algorithm, but it is common to use a CART model.
# 2. GBDT
The goal of boosting is to solve the following optimization problem
\begin{align}
&\underset{f}{min}\,L(f)=\underset{f}{min}\sum_{i=1}^Nl(y_i,f(\vec{x}_i)) \\
&f(\vec{x}_i)=w_0+\sum_{m=1}^Mw_mf_m(\vec{x}_i)
\end{align}
with the training data $\{\vec{x}_i,y_i\}_{i=1}^N$. We can view this problem as a optimization problem where $\vec{f}=\left(f(\vec{x}_1),f(\vec{x}_2),\cdots,f(\vec{x}_N)\right)$ are the "parameters". We will solve this stagewise, using gradient descent. At step m, let $\vec{g}_m$ be the gradient of $L(f)$ evaluate at $f=f_{m-1}$.
\begin{align}
&\vec{g}_m=\left(g_{1m},g_{2m},\cdots,g_{Nm}\right) \\
&g_{im}=\left[\frac{\partial l(y_i,f(\vec{x}_i)}{\partial f(\vec{x}_i)}\right]_{f=f_{m-1}}
\end{align}
Then the "parameters" $\vec{f}=\left(f(\vec{x}_1),f(\vec{x}_2),\cdots,f(\vec{x}_N)\right)$ are updated as
$$
\vec{f}_m=\vec{f}_{m-1}-\eta_m\vec{g}_m
$$
where $\eta_m$ is the step length.

Until now, the model is useless, since it only optimizes $\vec{f}$ at a fixed set of $N$ points. We need the function to be smooth enough so as to good generalize performance. The intuition of GBDT is to fitting a weak learner to approximate the negative gradient instead of using the gradient directly. This is the magic of GBDT. That is, we use the negative gradient to learn a tree.
$$
\underset{\gamma}{argmin}\sum_{i=1}^N\left(-g_{im}-\phi(\vec{x}_i;\gamma)\right)^2
$$
where $\gamma$ is the parameters of a tree.
# 3. Interpretation
In many applications it is useful to be able to interpret the derived approximation $\hat{f}(\vec{x})$. This involves gaining an understanding of those particular input variables that are most influential in contributing to its variation.

For a single decision tree $T$, Breiman et al.(1984) proposed
$$
\Upsilon_l^2(T)=\sum_{t=1}^{J-1}i_t^2\mathbb{1}(v(t)=l)
$$
as a measure of relevance for each feature $x_l$, in which $J$ is the number of the leaf node (The tree is always full binary tree) and the sum is over the $J-1$ internal nodes of the tree. At each such node $t$, one of the input variables $x_{v(t)}$ is used to partition the region associated with that node into two subregions; within each a separate constant is fit to the response values. The particular variable chosen is the one that gives maximal estimated improvement $i_t^2$ in squared error risk over that for a constant fit over the
entire region. The squared relative importance of variable $x_l$ is the sum of such squared improvements over all internal nodes for which it was chosen as the splitting variable.

This importance measure is easily generalized to additive tree expansions, in which it is simply averaged over the trees
$$
\Upsilon_l^2=\frac{1}{M}\sum_{m=1}^M\Upsilon_l^2(T_m)
$$
# 4. XGBoost
## 4.1 Regularized Learning Objective
For a given data set with $n$ examples and $m$ features $D=\{(\vec{x}_i,y_i)_{i=1}^N\}$, boosting tree algorithm predict the output as follows
$$
\hat{y}_i=\sum_{k=1}^Kf_k(\vec{x}_i), \quad f_k \in \mathcal{F}
$$
where $\mathcal{F}=\{f(\vec{x})=w_{q(\vec{x})}\}(q: \mathbb{R}^m \rightarrow T, \vec{w} \in \mathbb{R}^T)$ is the space of regression trees. Here $q$ represents the structure of each tree that maps an example to the corresponding leaf index $q(\vec{x})$. $T$ is the number of leaves in the tree. Each $f_k$ corresponds to an independent tree structure $q$ and leaf weight $\vec{w}$.

To learn the set of trees used in the model, we minimize the following regularized objective
\begin{align}
&\mathcal{L}(f_1,f_2,\cdots,f_K)=\sum_{i=1}^Nl(y_i,\hat{y}_i)+\sum_{k=1}^K\Omega(f_k) \\
&\Omega(f)=\gamma T+\frac{1}{2}\lambda\|\vec{w}\|^2 \\
\end{align}
The $\Omega$ term penalizes the complexity of the model (i.e., the regression tree functions). The additional regularization term helps to smooth the final learnt weights to avoid over-fitting.
## 4.2 Gradient Tree Boosting
The regularized learning objective is optimized in a greedy manner as the GBDT. Formally, let $\hat{y}_i^t$ be the predition of the $i$-th instance at the $t$-th iteration, we will need to add $f_t$ to minimize the following obejctive.
$$
\mathcal{L}^t(f_t)=\sum_{i=1}^Nl\left(y_i,\hat{y}_i^{t-1}+f_t(\vec{x}_i)\right)+\Omega(f_t)
$$
This means we greedily add the $f_t$ that most improves our model. Second-order approximation can be used to quickly optimize the objective in the genaral setting
$$
\mathcal{L}^t(f_t)\approx\sum_{i=1}^N\left[l(y_i,\hat{y}_i^{t-1})+g_if_t(\vec{x}_i)+\frac{1}{2}h_if_t^2(\vec{x}_i)\right]+\Omega(f_t)
$$
where 
\begin{align}
g_i &=\frac{\partial l(y_i,\hat{y}_i)}{\partial \hat{y}_i}|_{\hat{y}_i^{t-1}} \\
h_i &=\frac{\partial^2 l(y_i,\hat{y}_i)}{\partial^2 \hat{y}_i}|_{\hat{y}_i^{t-1}} \\
\end{align}
We can remove the constant terms to obtain the following simplified objective at step $t$
$$
\tilde{\mathcal{L}}^t(f_t)=\sum_{i=1}^N\left[g_if_t(\vec{x}_i)+\frac{1}{2}h_if_t^2(\vec{x}_i)\right]+\Omega(f_t)
$$
Define $I_j=\{i|q(\vec{x}_i=j\}$ as the instance set of leaf $j$. We can rewrite the equation above as follows
\begin{align}
\tilde{\mathcal{L}}^t(f_t) &=\sum_{i=1}^N\left[g_if_t(\vec{x}_i)+\frac{1}{2}h_if_t^2(\vec{x}_i)\right]+\gamma T+\frac{1}{2}\lambda\sum_{j=1}^T w_j^2  \\
                           &=\sum_{j=1}^T\left[(\sum_{i\in I_j}g_i)w_j+\frac{1}{2}(\sum_{i \in I_j}h_i+\lambda)w_j^2\right]+\gamma T
\end{align}
For a fixed structure $q(\vec{x})$, we can compute the optimal weight $\hat{w}_j$ of leaf $j$ by
$$
\hat{w}_j=\frac{\sum_{i\in I_j}g_i}{\sum_{i \in I_j}h_i+\lambda}
$$
and calculate the corresponding optimal value by
$$
\tilde{\mathcal{L}}^t(f_t)=-\frac{1}{2}\sum_{j=1}^T\frac{(\sum_{i\in I_j}g_i)^2}{\sum_{i \in I_j}h_i+\lambda}+\gamma T
$$
This equation can be used as a scoring function to measure the quality of a tree structure $q$.

Normally it is impossible to enumerate all the possible tree structures $q$. A greedy algorithm that starts from a single leaf and iteratively adds branches to the tree is used instead. Assume that $I_L$ and $I_R$ are the instance sets of left and right nodes after the split. Then the loss reduction after the split is given by
$$
\mathcal{L}_{split}=\frac{1}{2}\left[\frac{(\sum_{i\in I_L}g_i)^2}{\sum_{i \in I_L}h_i+\lambda}+\frac{(\sum_{i\in T_R}g_i)^2}{\sum_{i \in I_R}h_i+\lambda}-\frac{(\sum_{i\in I}g_i)^2}{\sum_{i \in I}h_i+\lambda}\right]-\gamma
$$
## 4.3 Shrinkage and Column Subsampling
Besides the regularized term in the objective function, two additional techniques are used to further prevent over-fitting. 
* **Shrinkage** which is also called learning rate in GBDT. Shrinkage scales newly added weights by a factor $\eta$ after each step of tree boosting. Shrinkage reduces the influence of
each individual tree and leaves space for future trees to improve the model.
* ** Column subsampling**. This technique is used to prevent over-fitting, as well as speeding up computations.

## 4.4 Split finding algorithm
### 4.4.1 Basic exact greedy algorithm
One of the key problems in tree learning is to find the best split.In order to do so, a split finding algorithm enumerates over all the possible splits on all the features, which is called *exact greedy algorithm*. It is computationally demanding to enumerate all the possible splits for continuous features. In order to do so efficiently, the algorithm must first pre-sort the data according to feature values and visit the data in sorted order to accumulate the gradient statistics for the structure score. The exact greedy algorithm is shown below. You'd better read [Mehta's paper](https://sci2s.ugr.es/keel/pdf/algorithm/congreso/SLIQ.pdf) for  implementation detail.
<img src="imgs/1.png" alt="drawing" width="400"/>
### 4.4.2 Approximate algorithm
The exact greedy algorithm is very computation intensive and is impossible to efficiently do so when the data does not fit entirely into memory. Same problem also arises in the distribution setting. An approximate algorithm is shown as follows which using bins for continuous features.
<img src="imgs/2.png" alt="drawing" width="400"/>

To summarize, the algorithm first proposes candidate splitting points according to percentiles of feature distribution. The algorithm then maps the continuous features into buckets split by these candidate points, aggregates the statistics and finds the best solution among proposals based on the aggregated statistics.

There are two variants of the algorithm, depending on when the proposal is given. The global variant proposes all the candidate splits during the initial phase of tree construction, and uses the same proposals for split finding at all levels. The local variant re-proposes after each split. The global method requires less proposal steps than the local method. However, usually more candidate points are needed for the global proposal because candidates are not refined after each split. The local proposal refines the candidates after splits,and can potentially be more appropriate for deeper trees.
### 4.4.3 Weighted Quantile Sketch
Now, we turn to the most important part of the approximate algorithm, which means candidate proposal. Usually percentiles of a feature are used to make candidates distribute evenly on the dataset. Formally, let multi-set $D_k=\{(x_{1k},h_1),(x_{2k},h_2),\cdots,(x_{Nk},h_N)\}$ represent the $k$-th feature values and second order gradient of each training instances. We can define a rank function $r_k: \mathbb{R} \rightarrow [0,+\infty)$ as
$$
r_k(z)=\frac{1}{\sum_{(x,h)\in D_k}h}\sum_{(x,h)\in D_k,x<z}h
$$
which represents the proportion of instances whose feature value k is smaller than $z$, weighted by the second order gradient. So the goal of the candidate proposal is to find  candidate split points $\{s_{k1},s_{k2},\cdots,s_{kl}\}$ such that
$$
|r_k(s_{k,j})-r_k(s_{k,j+1})|<\epsilon \quad  s_{k1}=\underset{i}{min}\,x_{ik},\quad s_{kl}=\underset{i}{max}\,x_{ik}
$$
Here $\epsilon$ is an approximation factor. Intuitively, this means that there is roughly $1/\epsilon$ candidate points because $r_k(s_{k1})=0\,\text{and}\,r_k(s_{kl})=1$.

The reason why using $h_i$ as the weight of each instance can be explained by rewrite the loss as
\begin{align}
\tilde{\mathcal{L}}^t(f_t) &=\sum_{i=1}^N\left[g_if_t(\vec{x}_i)+\frac{1}{2}h_if_t^2(\vec{x}_i)\right]+\gamma T+\frac{1}{2}\lambda\sum_{j=1}^T w_j^2 \\
                           &=\sum_{j=1}^N\left[\frac{1}{2}h_i\left(f(\vec{x}_i)+\frac{g_i}{h_i}\right)^2-\frac{g_i^2}{2h_i}\right]+\gamma T+\frac{1}{2}\lambda\sum_{j=1}^T w_j^2
\end{align}
which is exactly weighted squared loss with target $-g_i/h_i$ and weights $h_i$. This is the reason why $h_i$ is used as the weight of each instance.

In XGBoost, a novel distributed weighted quantile sketch algorithm was proposed. A detailed description of the algorithm can be referred to [this](https://homes.cs.washington.edu/~tqchen/pdf/xgboost-supp.pdf)

## 4.5 Missing value treatment
Missing value can be treated automatically in XGBoost by adding a default direction in each tree node. When a feature is missing in a data instance, the instance is classified into the default direction.There are two choices of default direction in each branch. The optimal default directions are learnt from the data as follows.The key point is to only visit the non-missing entries $I_k$.The presented algorithm treats the non-presence as a missing value and learns the best direction to handle missing values.
<img src="imgs/3.png" alt="drawing" width="400"/>
# 5.LightGBM
## 5.1 Gradient-based one-side sampling (GOSS, for data subsampling)
LightGBM was used in situations where feature dimension is high and data size is large. Gradient-based one-side sampling is used to tackle the problem of big data size.With GOSS, it exclude a significant proportion of data instances with small gradients, and only use the rest to estimate the information gain. Since the data instances with larger gradients play a more important role in the computation of information gain, GOSS can obtain quite accurate estimation of the information gain with a much smaller data size.

GOSS keeps all the instances with large gradients and performs random sampling on the instances with small gradients. In order to compensate the influence to the date distribution, when computing the information gain, GOSS introduces a constant multiplier for the data instances with small gradients.Specifically, GOSS firstly sorts the data instances according to the absolute value of their gradients and selects the top $a*100\%$ instances. Then it randomly samples $b*100\%$ instances from the rest of the data. After that, GOSS amplifies the sampled data with small gradients by a constant $\frac{1}{b}$ when calculating the information gain. By doing so, we put more focus on the under-trained instances without changing the original data distribution by much.
## 5.2 Exclusive feature bundling (EFB, for feature reduction)
High-dimensional data are usually very sparse. The sparsity of the feature space provides us a possibility of designing a nearly lossless approach to reduce the number of features.Specifically, in a sparse feature space, many features are mutually exclusive, i.e., they never take nonzero values simultaneously. We can safely bundle exclusive features into a single feature (which was called an exclusive feature bundle). By a carefully designed feature scanning algorithm, we can build the same feature histograms from the feature bundles as those from individual features. In this way, the complexity of histogram building changes from $O(\#data \times \#feature)$ to $O(\#data \times \#bundle)$, while $\#bundle<<\#feature$. Then we can significantly speed up the training of GBDT without hurting the accuracy.

There are two issures to be addressed. The first one is to determine which features should be bundled together. The second is how to construct the bundle. The implementation details can refer to the original [paper](https://papers.nips.cc/paper/6907-lightgbm-a-highly-efficient-gradient-boosting-decision-tree.pdf)